## **A basic Extract, Transform and Load (ETL) pipeline using web scrapping, pandas and sql**

### **Step 0: Importing Libraries**

In [9]:
from io import StringIO
import requests
from bs4 import BeautifulSoup
import pandas as pd
import sqlite3
from datetime import datetime

### **Step 1: Maintaining a Log File**
This step is done to record the logs while performing ETL and it is not neccessary in an ETL Pipeline

In [10]:
def log_progress(message):
    
    """This function, logs the mentioned message of a given stage of the
    code execution to a log file. Function returns nothing"""

    with open("./logs/code_logs.txt", 'a') as f:
        f.write(f"{datetime.now()}: {message}\n")

### **Step 2: Extract A Data from Website**

In [11]:
# First Extract the data through Website
url = 'https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks'

# Specify the target table we want to extract (table titled "By market capitalization")
table_attribs = 'By market capitalization'

# Send an HTTP request to the webpage and parse the HTML content using BeautifulSoup 
soup = BeautifulSoup(requests.get(url).text,"html.parser")

# Locate the required table by finding the heading text, 
# then navigating to the next <table> element after that heading
table = soup.find('span', string= table_attribs).find_next('table')

# Convert the extracted HTML table into a structured pandas DataFrame
df = pd.read_html(StringIO(str(table)))[0]

log_progress('Data extraction complete. Initiating Transformation process')

# show df
df

,Rank,Bank name,Market cap (US$ billion)
0,1,JPMorgan Chase,432.92
1,2,Bank of America,231.52
2,3,Industrial and Commercial Bank of China,194.56
3,4,Agricultural Bank of China,160.68
4,5,HDFC Bank,157.91
5,6,Wells Fargo,155.87
6,7,HSBC Holdings PLC,148.90
7,8,Morgan Stanley,140.83
8,9,China Construction Bank,139.82
9,10,Bank of China,136.81


### **Step 3: Transforming the Data**

In [ ]:
#  Loading csv files(exchange_rate.csv) to convert Market Cap values from USD into other currencies.

# Convert it into dictionary for quick key-based lookup of rates (e.g., "EUR", "GBP", "INR").
# 'index_col=0' sets the first column as the key, and we extract the "Rate" column
exchange_rateData = pd.read_csv("./input/exchange_rate.csv", index_col= 0).to_dict()["Rate"]

# Now convert the currencies using the exchange rates

# Add a new column: Market Cap converted from US$ to EUR (rounded to 2 decimals)
df["Market cap (EUR billion)"] = round(df["Market cap (US$ billion)"] * exchange_rateData["EUR"], 2)

# Add a new column: Market Cap converted from USD to GBP (rounded to 2 decimals)
df["Market cap (GBP billion)"] = round(df["Market cap (US$ billion)"] * exchange_rateData["GBP"], 2)

# Add a new column: Market Cap converted from USD to INR (rounded to 2 decimals)
df["Market cap (INR billion)"] = round(df["Market cap (US$ billion)"] * exchange_rateData["INR"], 2)

# Display the transformed DataFrame
df

,Rank,Bank name,Market cap (US$ billion),Market cap (EUR billion),Market cap (GBP billion),Market cap (INR billion)
0,1,JPMorgan Chase,432.92,419.93,346.34,36798.20
1,2,Bank of America,231.52,224.57,185.22,19679.20
2,3,Industrial and Commercial Bank of China,194.56,188.72,155.65,16537.60
3,4,Agricultural Bank of China,160.68,155.86,128.54,13657.80
4,5,HDFC Bank,157.91,153.17,126.33,13422.35
5,6,Wells Fargo,155.87,151.19,124.70,13248.95
6,7,HSBC Holdings PLC,148.90,144.43,119.12,12656.50
7,8,Morgan Stanley,140.83,136.61,112.66,11970.55
8,9,China Construction Bank,139.82,135.63,111.86,11884.70
9,10,Bank of China,136.81,132.71,109.45,11628.85


### **Step 4(a) Process of loading the data into CSV**

In [ ]:
# Create a folder named "output"(or with any other name), then export the file with file_name.csv and store it in avariable

transformed_Table = df.to_csv("./output/transformedData.csv")

### **Step 4(b) Process of loading the data into database**

In [ ]:
# Define database path and target table name for storing the data
db_path = "./output/Banks.db"
table_name = "By market capitalization"

# Establish a connection to the SQLite database (creates file if it doesn’t exist)
conn = sqlite3.connect(db_path)
cur = conn.cursor()

# Load the transformed DataFrame into SQLite database as a new table
df.to_sql(table_name, conn, if_exists= "replace", index=False)

# Query function to execute SQL queries on the database
def run_query(query_statement, conn):
    """ This function runs the query on the database table and
    prints the output on the terminal. Function returns nothing. """
    
    cursor = conn.cursor()
    # cursor = sql_connection.cursor()
    cursor.execute(query_statement)
    result = cursor.fetchall()

    return result

# Run SQL queries on the stored data and display the results
print(run_query('SELECT * FROM Largest_banks', conn))

print(run_query('SELECT AVG("Market cap (GBP billion)") FROM Largest_banks', conn))

print(run_query('SELECT "Bank name" FROM Largest_banks LIMIT 5', conn))